In [44]:
import pandas as pd


In [45]:
df=pd.read_csv('/Users/joe/Desktop/sql/data/realtor_data.csv')
df.head()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_dropped=df.drop('prev_sold_date',axis=1).copy()
df_dropped.drop_duplicates(inplace=True)


In [ ]:
df_dropped.dropna(axis=0,inplace=True)
df_dropped.info()
df_dropped.to_parquet('/Users/joe/Desktop/sql/data/cleaned_data.parquet',engine='fastparquet')


<class 'pandas.core.frame.DataFrame'>
Index: 1353936 entries, 0 to 2226381
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   brokered_by  1353936 non-null  float64
 1   status       1353936 non-null  object 
 2   price        1353936 non-null  float64
 3   bed          1353936 non-null  float64
 4   bath         1353936 non-null  float64
 5   acre_lot     1353936 non-null  float64
 6   street       1353936 non-null  float64
 7   city         1353936 non-null  object 
 8   state        1353936 non-null  object 
 9   zip_code     1353936 non-null  float64
 10  house_size   1353936 non-null  float64
dtypes: float64(8), object(3)
memory usage: 124.0+ MB


In [ ]:
# Sample the DataFrame
df_sampled = df_dropped.sample(n=1000000)

# Reset the index, drop the old index, and start the new index from 1
df_sampled = df_sampled.reset_index(drop=True)
df_sampled.index = df_sampled.index + 1

# Save the DataFrame to an Excel file
df_sampled.to_excel('/Users/joe/Desktop/sql/data/cleaned_data.xlsx', engine='xlsxwriter', index=True)


In [46]:
df_cleaned=df_sampled.groupby(['state','city'])['price'].agg('median').reset_index().copy()
df_cleaned

,state,city,price
0,Alabama,Abbeville,172250.0
1,Alabama,Adamsville,152450.0
2,Alabama,Addison,131950.0
3,Alabama,Adger,194900.0
4,Alabama,Akron,489950.0
...,...,...,...
24974,Wyoming,Wheatland,239750.0
24975,Wyoming,Wilson,4050000.0
24976,Wyoming,Worland,229900.0
24977,Wyoming,Wright,205000.0


In [47]:
df_filtered = df_cleaned[df_cleaned['price'] != 1.0]
max_price_city = df_filtered.loc[df_filtered.groupby('state')['price'].idxmax()]
min_price_city = df_filtered.loc[df_filtered.groupby('state')['price'].idxmin()]
df_max_min_prices= pd.concat([max_price_city, min_price_city]).sort_values(by='state').reset_index(drop=True)
df_max_min_prices.to_excel('/Users/joe/Desktop/sql/data/max_min_prices.xlsx',engine='xlsxwriter')